In [1]:
# import packages
import requests
import os
import json
import re

In [2]:
# import original tweet data
file1 = open("TwitterData.txt", "r")
textfile = file1.read()
tweets = textfile.splitlines()
while("" in tweets) :
    tweets.remove("")

In [3]:
# List to save slang percentage of each tweet
slangPerc = []
# LINUX kernel English dictionary
file = open("/usr/share/dict/words", "r")
# list of words in English dictionary
words = re.sub("[^\w]", " ",  file.read()).split()
# Compares words in tweet to English dictionary to identify slang
for i in range(len(tweets)):
    # count slang words in tweet
    count = 0
    # list of words in the tweet
    tweetWords = tweets[i].split()
    if len(tweetWords) != 0:
        # iterate through list to compare each word to English dictionary to check for slang
        for j in range(len(tweetWords)):
            word = tweetWords[j]
            if word.strip().lower() in words:
                continue
            else:
                count += 1
        slangPerc.append(count/len(tweetWords))

In [4]:
# list of re-translated tweets
file2 = open("TranslatedData.txt", "r")
textfile = file2.read()
translatedTweets = textfile.splitlines()
while("" in translatedTweets) :
    translatedTweets.remove("")

In [5]:
# List to save match percentage of each tweet
matchPerc = []
# Compares words in re-translated tweet to original tweet
for i in range(len(translatedTweets)):
    # list of words in original tweet
    words = tweets[i].split()
    # count matched words in tweet
    count = 0
    # list of words in the re-translated tweet
    tweetWords = translatedTweets[i].split()
    # iterate through list to compare each word to English dictionary to check for slang
    for j in range(len(tweetWords)):
        try:
            word1 = tweetWords[j]
            word2 = words[j]
            if word1.strip().lower() == word2.strip().lower():
                count += 1
        except:
            continue
    matchPerc.append(count/len(tweetWords))

[0.4,
 0.375,
 0.6470588235294118,
 0.6,
 0.46153846153846156,
 0.0,
 0.23076923076923078,
 0.5555555555555556,
 0.07142857142857142,
 1.0,
 0.46153846153846156,
 0.38461538461538464,
 0.8,
 0.0,
 0.28125,
 0.20833333333333334,
 0.8,
 0.3225806451612903,
 0.5,
 0.2619047619047619,
 0.15151515151515152,
 0.0,
 0.11363636363636363,
 0.2,
 0.05263157894736842,
 0.3076923076923077,
 0.11627906976744186,
 0.20833333333333334,
 0.0967741935483871,
 0.06818181818181818,
 0.0,
 0.21428571428571427,
 0.0,
 0.4,
 0.0,
 1.0,
 0.5,
 1.0,
 0.5,
 1.0,
 0.0,
 0.5,
 1.0,
 0.0,
 0.06896551724137931,
 0.2727272727272727,
 0.1111111111111111,
 0.2727272727272727,
 0.42857142857142855,
 0.3333333333333333,
 0.5,
 0.25,
 0.14285714285714285,
 1.0,
 0.26666666666666666,
 0.15789473684210525,
 0.21052631578947367,
 0.1111111111111111,
 0.0,
 0.26666666666666666,
 0.125,
 0.23076923076923078,
 0.0,
 0.4444444444444444,
 0.35714285714285715,
 0.5,
 0.125,
 0.15789473684210525,
 0.2222222222222222,
 0.090909090